In [ ]:
#| default_exp camera

# Camera 

> Camera model focusing on a model of a pin hole camera with distortions

Need to check (and possible integrate) the [code from pixlib](https://github.com/cvg/pixloc/blob/master/pixloc/pixlib/geometry/wrappers.py#L224) that provides and camera model both for torch and numpy. The discussoions in [this tweet](https://twitter.com/pesarlin/status/1558087424445652998?s=27&t=ILeCIQvS1YTIJL8yMhYFMg) anre also useful. See [pinhole model in kornia](https://kornia.readthedocs.io/en/latest/geometry.camera.pinhole.html) - no distortions and use 4x4 homogenous representation all the way also with batch dimensions.

Suggest a Pose API, Intrinsics API and camera API that include both.

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
from fastcore.utils import *    # to get patch

In [ ]:
#|export

import numpy as np
import cv2
from easydict import EasyDict as edict

# Geometric camera model

The class `Intrinsicts` models a pinhole camera with distortions. Useful books are:

* The book [Computer Vision: Algorithms and Applications, 2nd ed.](http://szeliski.org/Book/),  [Richard Szeliski](https://szeliski.org/RichardSzeliski.htm), 2022 that can be freely downloaded.
* [Multiple View Geometry in Computer Vision, 2nd ed.](https://www.robots.ox.ac.uk/~vgg/hzbook/), Hartley and Zisserman, , Cambridge University Press, 2004
* [An Invitation to 3-D Vision (pdf)](https://www.eecis.udel.edu/~cer/arv/readings/old_mkss.pdf), Yi Ma, Jana Koˇseck´a, Stefano Soatto, Shankar Sastry, 2001


In [ ]:
#|export

SUPPORTED_CAMERA_MODELS = dict(
    SIMPLE_PINHOLE = dict(id=0, n_params=3, params_str='f, cx, cy'), 
    PINHOLE        = dict(id=1, n_params=4,params_str='fx, fy, cx, cy'), 
    SIMPLE_RADIAL  = dict(id=2, n_params=4,params_str='f, cx, cy, k'), 
    RADIAL         = dict(id=3, n_params=5,params_str='f, cx, cy, k1, k2'), 
    OPENCV         = dict(id=4, n_params=8,params_str='fx, fy, cx, cy, k1, k2, p1, p2'), 
    OPENCV_FISHEYE = dict(id=5, n_params=8,params_str='fx, fy, cx, cy, k1, k2, k3, k4'), 
    FULL_OPENCV    = dict(id=6, n_params=12,params_str='fx, fy, cx, cy, k1, k2, p1, p2, k3, k4, k5, k6'), 
    FOV            = dict(id=7, n_params=5,params_str='fx, fy, cx, cy, omega'), 
    OPENCV5        = dict(id=-1, n_params=9,params_str='fx, fy, cx, cy, k1, k2, p1, p2, k3'),
    UNKNOWN        = dict(id=-1, n_params=0,params_str='[]'), 
)


In [ ]:
#|export

class Intrinsicts:
    'Camera intrinsic model'
    def __init__(self, 
                 camera_model_name: str,   # One of the keys in SUPPORTED_CAMERA_MODELS
                 width: int,               # width of the image in pixels
                 height: int,              # height of the image in pixels
                 params: list):            # parameters, in COLMAP conventions
        # prior_focal_length : 1 if we have confidence in the modelparameters and 0 if we do not trust the model parameters

        if camera_model_name not in SUPPORTED_CAMERA_MODELS:
            raise ValueError(f'Camera model ["{camera_model_name}"] not recognized as colmap camera model')
        
        param_names = SUPPORTED_CAMERA_MODELS[camera_model_name]['params_str'].split(',')
        param_names = [p.strip() for p in param_names]
        if len(param_names) != len(params):
            raise ValueError(f'{camera_model_name} expectes {len(param_names)} parameters but got {len(params)}') 

        self._w = width
        self._h = height

        self._camera_model_name = camera_model_name
        self._set_params(camera_model_name, params)

    @staticmethod
    def supported_camera_models():
        print('List of supported camera models and their parameters')
        print(55*'_')
        for m in SUPPORTED_CAMERA_MODELS:
            p = SUPPORTED_CAMERA_MODELS[m]['params_str']
            print(f'{m:20}: {p}')


    def __str__(self):
        s  = f'Camera: {self.camera_model_name()}\n'
        s += f'  w,h={self.width(),self.height()}\n'
        s += f'  params: {self.get_params()}\n'
        s += f'  cx,cy= ({self.cx()},{self.cy()})\n'
        s += f'  fx,fy= ({self.fx()},{self.fy()})\n'
        s += f'  distortions: {self.distortions()}\n'


        return s

    __repr__ = __str__

    @staticmethod
    def from_pinhole_model(fx: float,   # Focal length (x) in pixels
                           fy: float,   # Focal length (y) in pixels. fy might be equal to fx (SIMPLE_PINHOLE model) or different (PINHOLE model)
                           cx:float,    # Camera center (x) in pixels
                           cy: float,   # Camera center (y) in pixels
                           width: int,  # Image width in pixels
                           height: int  # Image height in pixels
                           ) -> 'Intrinsicts':
        'Contructing camera intrinsics model from opencv compatible data'
        if fx == fy:
            camera_model_name = 'SIMPLE_PINHOLE'
            params = [fx, cx, cy]
        else:
            camera_model_name = 'PINHOLE'
            params = [fx, fy, cx, cy]

        return Intrinsicts(camera_model_name,width, height, params)


    @staticmethod
    def from_opencv_model(K: np.ndarray, # 3x3 camera matrix
                          distortions: np.ndarray, # distortion array as produced by OpenCv
                          width: int, # Camera width in pixels
                          height: int # Camera height in pixels
                         ) -> 'Intrinsicts':
        'Contructing camera intrinsics model from opencv compatible data'
        if not isinstance(distortions, list):
            if len(distortions.shape) == 2:
                distortions = distortions.squeeze()
            distortions= distortions.tolist()
     
        fx = K[0,0]
        cx = K[0,2]
        fy = K[1,1]
        cy = K[1,2]

        params = [fx, fy, cx, cy]
        if len(distortions) == 4:
            camera_model_name = 'OPENCV'
            params += distortions
        elif len(distortions) == 5:
            camera_model_name = 'OPENCV5'
            params += distortions
        else:
            raise ValueError(f'Do not support opencv model with {len(distortions)} parameters')

        return Intrinsicts(camera_model_name,width, height, params)

    @staticmethod
    def from_test_model():
        'Contructing camera intrinsics model from opencv calibration tutorial'
        w, h = 640, 480 

        distortions = np.array(
            [
            [-2.6637260909660682e-01], 
            [-3.8588898922304653e-02], 
            [1.7831947042852964e-03], 
            [-2.8122100441115472e-04], 
            [2.3839153080878486e-01]
            ]
        )

        mtx = np.array(
            [
                [5.3591573396163199e+02, 0.,                     3.4228315473308373e+02],
                [0.,                     5.3591573396163199e+02, 2.3557082909788173e+02],
                [0.,                     0.,                     1.]
            ]
        )

        return Intrinsicts.from_opencv_model(mtx,distortions,w, h)

    def camera_model_name(self) -> str:
        'Returns the name of the camera model, e.g. `OPENCV`'
        return self._camera_model_name

    def fx(self):
        'Returns the (x) focal point in pixels'
        return self._K[0,0]

    def fy(self):
        'Returns the (y) forcal point in pixels'
        return self._K[1,1]

    def cx(self):
        'Returns the x coordinate of the camera center in pixels'
        return self._K[0,2]

    def cy(self):
        'Returns the y coordinate of the camera center in pixels'
        return self._K[1,2]

    def w(self):
        'Returns the width of image, same as calling to the `width` method'
        return self._w

    def width(self):
        'Returns the width of image, same as calling to the `w` method'
        return self._w

    def h(self):
        'Returns the height of image, same as calling to the `height` method'
        return self._h

    def height(self):
        'Returns the height of image, same as calling to the `h` method'
        return self._h

    def is_single_focal_lenght(self):
        return 'SIMPLE' in self.camera_model_name()

    def K(self, 
          use_homogenous_coordinates:bool=True # If True, returns 4x4 camera matrix in homogenous coordinates, otherwise return a 3x3 camera matrix
          ) -> np.ndarray:

        if not use_homogenous_coordinates:
            return self._K[:3,:3]
        return self._K

    def distortions(self) -> np.ndarray:
        'Returns 1D distortion array'
        return self._distortions

    def get_params(self) -> list:
        'Get list of parametes as expected in the consrtructor for the given camera model'
        if self.is_single_focal_lenght():
            cp = [self.fx(), self.cx(), self.cy()]
        else:
            cp = [self.fx(), self.fy(), self.cx(), self.cy()]

        p = cp + [float(d) for d in self.distortions()]
        return p
  
    def camera2image(self, pc):
        p_camera_plane_undistorted = self.project_perspective(pc)
        p_camera_plane_distorted = self.distort(p_camera_plane_undistorted)
        p_image = self.camera_plane_distorted_to_image_plane(p_camera_plane_distorted)
        return p_image

    def project_perspective(
        self, 
        pc: np.array   # a 3D point in th ecamera frame, either with 3 components or with 4 components if expressed in homogenous coordinates
        ) -> np.ndarray:
        'A projective transformation cinverting a 3D point in canmera frame to a 2D point in the image plane of that frame'
        is_h = len(pc) == 4

        disparity = 1/pc[2]
        xc = pc[0]*disparity
        yc = pc[1]*disparity

        if is_h:
            return np.array([xc,yc, disparity])   # homogenous coordinates

        return np.array([xc,yc]), disparity


    def project_camera_plane_to_3d(self, pc: np.array, disparity=None):
        """project a 3D vector
           See Szeliski, 2.1

        Args:
            pw (np.array): 4D vector of a world 3D point 
        """

        pc_3d = np.array([pc[0]/disparity, pc[1]/disparity,1./disparity,1])

        p_3d = np.linalg.inv(self.extrinsics) @ pc_3d

        return p_3d


    def distort(self, p_cam_distorted: np.ndarray):
        # see line 888 in https://github.com/colmap/colmap/blob/dev/src/base/camera_models.h
        camera_model_name = self.camera_model_name()
        distortions = self.distortions()
        if len(distortions) == 0:
            return  p_cam_distorted.copy()


        if camera_model_name == 'OPENCV5':
            # See https://learnopencv.com/understanding-lens-distortion/
            k1 = distortions[0]
            k2 = distortions[1]
            p1 = distortions[2]
            p2 = distortions[3] 
            k3 = distortions[4]

            xd = p_cam_distorted[0]
            yd = p_cam_distorted[1]

            x2 = xd*xd
            y2 = yd*yd
            xy = xd*yd
            r2 = x2 + y2
            r4 = r2*r2
            r6 = r2*r4

            a = 1.0 + k1*r2  + k2*r4 + k3*r6
            xu = a*xd + 2.0*p1*xy + p2*(r2 + 2.0*x2)
            yu = a*yd + p1*(r2+2.0*y2) + 2.0*p2*xy
    
        p_cam_distorted = np.array([xu,yu])

        return p_cam_distorted

    def camera_plane_distorted_to_image_plane(
        self,
        pc_distorted: np.ndarray  # The distorted point in the reference camera plane 2D or 3D if in homogenous coordinates
        ) -> np.ndarray:
        'Transform an image given in the image plane of the camera reference frame into pixel coordinates in the image'
 
        is_h = False
        if len(pc_distorted) == 2:
            # to homogenous coordinates
            is_h = True
            pc_distorted = np.append(pc_distorted, 1.0)
        
        pix = self.K(use_homogenous_coordinates=False) @ pc_distorted

        if not is_h:
            return pix[:2]

        return pix


    def get_undistort_matrix(self, alpha=1.0):
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(self.K, self.distortions, (self.w,self.h), alpha, (self.w,self.h))
        return newcameramtx

    def _set_params(self, camera_model_name, params):
        param_names = SUPPORTED_CAMERA_MODELS[camera_model_name]['params_str'].split(',')
        param_names = [p.strip() for p in param_names]
        if len(param_names) != len(params):
            raise ValueError(f'{camera_model_name} expectes {len(param_names)} parameters but got {len(params)}') 

        self._params = params
        
        # First names should be one of f,fx,fy,cx,cy
        camera_matrix_components = ['f','fx','fy','cx','cy']
        dlist = []
        cp = edict(fx=0.,fy=0.,cx=0.,cy=0.)
        for i, (name, val) in enumerate(zip(param_names,params)):
#             print(name,val)
            if name not in camera_matrix_components:
                dlist.append(val)
            elif name == 'f': 
                cp.fx = val
                cp.fy = val
            else:
                cp[name] = val

        self._K = np.array(
            [
                [cp.fx, 0.0,     cp.cx,     0.0],
                [0.0,     cp.fy, cp.cy,     0.0],
                [0.0,     0.0,     1.0,     0.0 ],
                [0.0,     0.0,     0.0,     1.0 ]
            ]
        )
    
        self._distortions = np.array(dlist, dtype=np.float64)

    
    def _get_params_to_new_cx_cy_fx_fy(self, new_cx, new_cy, new_fx=None, new_fy=None):
        new_fx = self.fx if new_fx is None else new_fx
        new_fy = self.fy if new_fy is None else new_fy

        if self.is_single_focal_lenght():
            cp = [new_fx, new_cx, new_cy]
        else:
            cp =  [new_fx, new_fy, new_cx, new_cy]

        p = cp + [float(d) for d in self.distortions()]
        return p

    def crop_bbox(self, in_bbox, c_bbox, new_width, new_height):
        if in_bbox is None: return None

        if in_bbox.minx >= c_bbox.maxx or in_bbox.maxx <= c_bbox.minx: return None
        if in_bbox.miny >= c_bbox.maxy or in_bbox.maxy <= c_bbox.miny: return None

        b_xmin = max(in_bbox.minx - c_bbox.minx,0)
        b_xmax = min(max(in_bbox.maxx - c_bbox.minx,0), new_width)
        b_ymin = max(in_bbox.miny - c_bbox.miny,0)
        b_ymax = min(max(in_bbox.maxy - c_bbox.miny,0), new_height)
        if b_xmin == b_xmax or b_ymin == b_ymax: return None

        return edict(
            minx = b_xmin,
            maxx = b_xmax,
            miny = b_ymin,
            maxy = b_ymax
        )
    

    def resize(self, new_size):
        """Change camera intrinsicts due to image resize --> scale of focal lenghts
        Args:
            new_size (tuple): (destination_width, destination_height)
        """
        new_width = new_size[0]
        new_height = new_size[1]
        scale_w = new_width / self.width
        scale_h = new_height / self.height

        fx  = self.fx * scale_w    # fx
        cx  = self.cx * scale_w    # cx
        fy  = self.fy * scale_h    # fy
        cy  = self.cy * scale_h    # cy

        new_params = self._get_params_to_new_cx_cy_fx_fy(cx, cy, fx, fy)

        return Intrinsicts(
            camera_model_name=self.camera_model_name, 
            width=new_width, 
            height=new_height, 
            params=new_params
        )

    def crop(self, bbox, new_name=None):
        """Change camera intrinsicts due to clipping to a rectangular window --> shifting the proincipal point
        Args:
            min_crop_x (float): Minimal coordinate of clipping rectangle in x directior, in pixels
            min_crop_y ([float]): Minimal coordinate of clipping rectangle in y directior, in pixels
        """
        new_cx = self.cx -  int(round(bbox.minx))   # cx
        new_cy = self.cy - int(round(bbox.miny))   # cy

        new_width = bbox.maxx - bbox.minx
        new_height = bbox.maxy - bbox.miny

        new_params = self._get_params_to_new_cx_cy_fx_fy(new_cx, new_cy)

        return Intrinsicts(
            camera_model_name=self.camera_model_name, 
            width=new_width, 
            height=new_height, 
            params=new_params
        )

    def get_optimal_new_camera_matrix(self, alpha):
        """_summary_

        Args:
            alpha (float): A number between 0 and 1, If the value is 
               alpha = 0 --> when all the pixels in the undistorted image are valid
               alpha = 1 --> when all the source image pixels are retained in the undistorted image but with many black pixels 
            new_image_size (_type_, optional): _description_. Defaults to None.
        """
        # See cvGetOptimalNewCameraMatrix in line 2714 of https://github.com/opencv/opencv/blob/4.x/modules/calib3d/src/calibration.cpp
        # See https://docs.opencv.org/3.3.0/dc/dbb/tutorial_py_calibration.html

        outer, inner = self.icv_get_rectangles()

        new_image_width = self.width
        new_image_height = self.height
   
        # Projection mapping inner rectangle to viewport
        fx0 = (new_image_width-1)/ inner.width
        fy0 = (new_image_height-1)/ inner.height
        cx0 = -fx0 * inner.x
        cy0 = -fy0 * inner.y

        # Projection mapping outer rectangle to viewport
        fx1 = (new_image_width-1)/ outer.width
        fy1 = (new_image_height-1)/ outer.height
        cx1 = -fx1 * outer.x
        cy1 = -fy1 * outer.y

        # Interpolate between the two optimal projections
        fx = fx0*(1 - alpha) + fx1*alpha
        fy = fy0*(1 - alpha) + fy1*alpha
        cx = cx0*(1 - alpha) + cx1*alpha
        cy = cy0*(1 - alpha) + cy1*alpha

        new_params = [fx,fy,cx,cy]
        return Intrinsicts(
            camera_model_name='PINHOLE', 
            width=new_image_width, 
            height=new_image_height, 
            params=new_params
        )




    def project_image_plane_to_camera_plane(self, pu: np.ndarray):
        K = self.K(use_homogenous_coordinates=False)
        fx = K[0,0]
        fy = K[1,1]
        cx = K[0,2]
        cy = K[1,2]
        
        xc = (pu[0] - cx)/fx
        yc = (pu[1] - cy)/fy
        return np.array([xc, yc])

    def undistort(self, pc_distorted: np.array):
        # see line 565 in https://github.com/colmap/colmap/blob/dev/src/base/camera_models.h
        eps =np.finfo(np.float64).eps

        kNumIterations = 100
        kMaxStepNorm = np.float32(1e-10)
        kRelStepSize = np.float32(1e-6)

        J = np.eye(2)
        x0 = pc_distorted.copy()
        x = pc_distorted.copy()
        for i in range(kNumIterations):
            step0 = np.max([eps, kRelStepSize * x[0]])
            step1 = np.max([eps, kRelStepSize * x[1]])

            dx = self.distort(x)

            dx_0b = self.distort(np.array([x[0] - step0, x[1]]))
            dx_0f = self.distort(np.array([x[0] + step0, x[1]]))
            dx_1b = self.distort(np.array([x[0]        , x[1] - step1]))
            dx_1f = self.distort(np.array([x[0]        , x[1] + step1]))
            J[0, 0] = 1 + (dx_0f[0] - dx_0b[0]) / (2 * step0)
            J[0, 1] = (dx_1f[0] - dx_1b[0]) / (2 * step1)
            J[1, 0] = (dx_0f[1] - dx_0b[1]) / (2 * step0)
            J[1, 1] = 1 + (dx_1f[1] - dx_1b[1]) / (2 * step1)
    
            step_x = np.linalg.inv(J) @ (dx - x0)
            x -= step_x

            squaren_norm = step_x[0]*step_x[0] + step_x[1]*step_x[1]
            if squaren_norm < kMaxStepNorm:
                break

        return  x   # undistorted


    def icv_get_rectangles(self):
        # see icvGetRectangles, line 2460 in https://github.com/opencv/opencv/blob/4.x/modules/calib3d/src/calibration.cpp
        N = 9
        x_step = self.w / (N-1)
        y_step = self.h / (N-1)

        # Get a grid over [w,h] image in original, distorted, coordinates
        xu = []
        yu = []
        xu_left = []
        xu_right = []
        yu_bottom = [] 
        yu_top = []         
        for y in range(N):
            yp = y*y_step
            for x in range(N):
                xp = x*x_step
                ps = np.array([xp,yp])
                pc_distorted = self.project_image_plane_to_camera_plane(ps)                # from 2d pixel coordinates to 2D camera plane
                pc_undistorted = self.undistort(pc_distorted)                              # undistors with inverse lens distortions

                x_undistorted = pc_undistorted[0]
                y_undistorted = pc_undistorted[1]

                xu.append(x_undistorted)
                yu.append(y_undistorted)
                if x == 0: xu_left.append(x_undistorted)
                if x == N-1: xu_right.append(x_undistorted)
                if y == 0: yu_top.append(y_undistorted)
                if y == N-1: yu_bottom.append(y_undistorted)


        xmin_o = np.min(xu)
        xmax_o = np.max(xu)
        ymin_o = np.min(yu)
        ymax_o = np.max(yu)
        outer = edict(x=xmin_o, y=ymin_o, width=xmax_o-xmin_o, height=ymax_o-ymin_o)


        xmin_i = np.max(xu_left)
        xmax_i = np.min(xu_right)
        ymin_i = np.max(yu_top)
        ymax_i = np.min(yu_bottom)
        inner = edict(x=xmin_i, y=ymin_i, width=xmax_i-xmin_i, height=ymax_i-ymin_i)

        return outer, inner

    def get_fov(self):
        # Zeliltsky 2.60
        fovx = 2 * np.rad2deg(np.arctan2(self.width , (2 * self.fx)))
        fovy = 2 * np.rad2deg(np.arctan2(self.height , (2 * self.fy)))

        return edict(fovx=fovx, fovy=fovy)

    def to_dict(self):
        return self.as_dict()

    def as_dict(self):
        asdict = dict(
            width=self.width,
            height=self.height,
            camera_model_name=self.camera_model_name,
            params=[float(p) for p in self.params.tolist()]
        )
        return asdict

#     def to_json(self, json_file):
#         write_json_file(self.as_dict(), json_file)


### Supported camera models
This is the list of all supported camera models. Each item includes tha name of the camera model, e.g. `SIMPLE_RADIAL` and list of expected parameters as handled by `Intrinsicts`. All camera models of [COLMAP](https://github.com/colmap/colmap/blob/master/src/base/camera_models.h) are supported and some more, as needed.


In [ ]:
Intrinsicts.supported_camera_models()

List of supported camera models and their parameters
_______________________________________________________
SIMPLE_PINHOLE      : f, cx, cy
PINHOLE             : fx, fy, cx, cy
SIMPLE_RADIAL       : f, cx, cy, k
RADIAL              : f, cx, cy, k1, k2
OPENCV              : fx, fy, cx, cy, k1, k2, p1, p2
OPENCV_FISHEYE      : fx, fy, cx, cy, k1, k2, k3, k4
FULL_OPENCV         : fx, fy, cx, cy, k1, k2, p1, p2, k3, k4, k5, k6
FOV                 : fx, fy, cx, cy, omega
OPENCV5             : fx, fy, cx, cy, k1, k2, p1, p2, k3
UNKNOWN             : []


## Construct camera model

### Initialization
Call to the constructor of the class with parameters such as

In [ ]:
c = Intrinsicts(
    camera_model_name='OPENCV5',
    width=640,
    height=320,
    params=[400, 410.0, 320, 160,14,15,16,17,228]
)

c

Camera: OPENCV5
  w,h=(640, 320)
  params: [400.0, 410.0, 320.0, 160.0, 14.0, 15.0, 16.0, 17.0, 228.0]
  cx,cy= (320.0,160.0)
  fx,fy= (400.0,410.0)
  distortions: [ 14.  15.  16.  17. 228.]

### Construction functions 


In [ ]:
show_doc(Intrinsicts.from_opencv_model)

---

#### Intrinsicts.from_opencv_model

>      Intrinsicts.from_opencv_model (K:numpy.ndarray,
>                                     distortions:numpy.ndarray, width:int,
>                                     height:int)

Contructing camera intrinsics model from opencv compatible data

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| K | ndarray | 3x3 camera matrix |
| distortions | ndarray | distortion array as produced by OpenCv |
| width | int | Camera width in pixels |
| height | int | Camera height in pixels |
| **Returns** | **Intrinsicts** |  |

In [ ]:
show_doc(Intrinsicts.from_pinhole_model)

---

#### Intrinsicts.from_pinhole_model

>      Intrinsicts.from_pinhole_model (fx:float, fy:float, cx:float, cy:float,
>                                      width:int, height:int)

Contructing camera intrinsics model from opencv compatible data

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| fx | float | Focal length (x) in pixels |
| fy | float | Focal length (y) in pixels. fy might be equal to fx (SIMPLE_PINHOLE model) or different (PINHOLE model) |
| cx | float | Camera center (x) in pixels |
| cy | float | Camera center (y) in pixels |
| width | int | Image width in pixels |
| height | int | Image height in pixels |
| **Returns** | **Intrinsicts** |  |

As an example, consider the construction from an OpenCv model with 5 parameters:

In [ ]:
width = 1280
height = 720
fx = 600
fy = 600
cx = 1280 /2
cy = 720 /2
K = np.array(
    [
        [fx, 0.0, cx],
        [0.0, fy, cy],
        [0.0,0.0,1.0]
    ]
)


distortions = [-1.51960304e-01,  5.60700273e-01, -1.28234990e-02,  1.41775450e-03, 5.23404322e+00]
Intrinsicts.from_opencv_model(K,distortions,width, height)

Camera: OPENCV5
  w,h=(1280, 720)
  params: [600.0, 600.0, 640.0, 360.0, -0.151960304, 0.560700273, -0.012823499, 0.0014177545, 5.23404322]
  cx,cy= (640.0,360.0)
  fx,fy= (600.0,600.0)
  distortions: [-1.51960304e-01  5.60700273e-01 -1.28234990e-02  1.41775450e-03
  5.23404322e+00]

In [ ]:
show_doc(Intrinsicts.from_test_model)

---

#### Intrinsicts.from_test_model

>      Intrinsicts.from_test_model ()

Contructing camera intrinsics model from opencv calibration tutorial

The test model is taken form the [OpenCV calibration tutorial](https://docs.opencv.org/3.3.0/dc/dbb/tutorial_py_calibration.html) and the images used for calibrating the test model are given in  [this directory](https://github.com/opencv/opencv/tree/master/samples/data)

In [ ]:
Intrinsicts.from_test_model()

Camera: OPENCV5
  w,h=(640, 480)
  params: [535.915733961632, 535.915733961632, 342.28315473308373, 235.57082909788173, -0.2663726090966068, -0.03858889892230465, 0.0017831947042852964, -0.0002812210044111547, 0.23839153080878486]
  cx,cy= (342.28315473308373,235.57082909788173)
  fx,fy= (535.915733961632,535.915733961632)
  distortions: [-0.26637261 -0.0385889   0.00178319 -0.00028122  0.23839153]

## Access functions 

In [ ]:
print(f'Camera : {c.camera_model_name()}, fx={c.fx()}, fy={c.fy()}, cx={c.cx()}, cy={c.cy()}, distortions={c.distortions()}')
print(f'Camera width: {c.w()}, heigth: {c.h()}')

Camera : OPENCV5, fx=400.0, fy=410.0, cx=320.0, cy=160.0, distortions=[ 14.  15.  16.  17. 228.]
Camera width: 640, heigth: 320


In [ ]:
show_doc(Intrinsicts.K)

---

#### Intrinsicts.K

>      Intrinsicts.K (use_homogenous_coordinates:bool=True)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| use_homogenous_coordinates | bool | True | If True, returns 4x4 camera matrix in homogenous coordinates, otherwise return a 3x3 camera matrix |
| **Returns** | **ndarray** |  |  |

In [ ]:
print(f'Homogenous camera matrix: \n {c.K(use_homogenous_coordinates=True)}, {c.K(use_homogenous_coordinates=True).dtype}')
print(f'Non homogenous camera matrix: \n {c.K(use_homogenous_coordinates=False)}, {c.K(use_homogenous_coordinates=False).dtype}')

Homogenous camera matrix: 
 [[400.   0. 320.   0.]
 [  0. 410. 160.   0.]
 [  0.   0.   1.   0.]
 [  0.   0.   0.   1.]], float64
Non homogenous camera matrix: 
 [[400.   0. 320.]
 [  0. 410. 160.]
 [  0.   0.   1.]], float64


In [ ]:
show_doc(Intrinsicts.distortions)

---

#### Intrinsicts.distortions

>      Intrinsicts.distortions ()

Returns 1D distortion array

In [ ]:
print(f'Distortions: {c.distortions()}, {c.distortions().dtype}')

Distortions: [ 14.  15.  16.  17. 228.], float64


In [ ]:
show_doc(Intrinsicts.get_params)

---

#### Intrinsicts.get_params

>      Intrinsicts.get_params ()

Get list of parametes as expected in the consrtructor for the given camera model

In [ ]:
print(f'Parameters of camera model {c.camera_model_name()}: {c.get_params()}')

Parameters of camera model OPENCV5: [400.0, 410.0, 320.0, 160.0, 14.0, 15.0, 16.0, 17.0, 228.0]


## Project and unproject 

## Examples

The data is taken from the [OpenCV calibration tutorial](https://docs.opencv.org/3.3.0/dc/dbb/tutorial_py_calibration.html) and was copied from [this directory](https://github.com/opencv/opencv/tree/master/samples/data)

In [ ]:
import os
import sys
from pathlib import Path
import cv2


def get_test_images():
    root_dir = Path(os.getcwd())
    if str(root_dir) not in sys.path:
        sys.path.insert(0,str(root_dir))
    else:
        print('PATH O.K.')

    img_dir = root_dir / 'data' 
    img_file = img_dir / 'left03.jpg'
    img = cv2.imread(str(img_file))
    h,  w = img.shape[:2]
    print(w, h)



ci = Intrinsicts.from_test_model()
ci

Camera: OPENCV5
  w,h=(640, 480)
  params: [535.915733961632, 535.915733961632, 342.28315473308373, 235.57082909788173, -0.2663726090966068, -0.03858889892230465, 0.0017831947042852964, -0.0002812210044111547, 0.23839153080878486]
  cx,cy= (342.28315473308373,235.57082909788173)
  fx,fy= (535.915733961632,535.915733961632)
  distortions: [-0.26637261 -0.0385889   0.00178319 -0.00028122  0.23839153]

In [ ]:

# p_camera = np.array([0.5,0.5,1.0])
# pimage = ci.camera2image(p_camera)
# print(pimage)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()